In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import cv2
import numpy as np
import csv
from sklearn.utils import shuffle
from pylab import *
from matplotlib import gridspec
import datetime
from pathlib import Path
cdict = {'red': ((0.0, 1.0, 1.0),
                 (0.125, 1.0, 1.0),
                 (0.25, 1.0, 1.0),
                 (0.5625, 1.0, 1.0),
                 (1.0, 0.0, 0.0)),
         'green': ((0.0, 0.0, 0.0),
                   (0.25, 0.0, 0.0),
                   (0.5625, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),
         'blue': ((0.0, 0.0, 0.0),
                  (0.5, 0.0, 0.0),
                  (1.0, 0.0, 0.0))}
my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)

from sklearn.utils import shuffle
%matplotlib inline

#datapath = '/home/microway/Shuo/CarND/CarND-BehaviorCloning-Project/data-given/'
#data = pd.read_csv('/Users/Shuo/study/Project-predictive_study/data_2016_1t_dir1.csv',delimiter=',')

In [2]:
data = []
waze = pd.read_csv('Waze2016_I235.csv',delimiter=',')
timewaze = pd.to_datetime(waze['start'],format='%m/%d/%Y %H:%M:%S')
weather = pd.read_csv('weather_1min.csv',delimiter=',')
timeweather = pd.to_datetime(weather['time'],format='%m/%d/%Y %H:%M:%S')

In [3]:


def wave_window_get(wave,milemarker,wavemin,direction):
    wave_window = wave[(wave['MM']<milemarker+0.001-space_shift) \
                       & (wave['MM']>milemarker+0.001-space_window_size_mi-space_shift) \
                       & (wave['min']<wavemin+0.1-time_shift) \
                       & (wave['min']>wavemin+0.1-time_window_size_min-time_shift) \
                       & (wave['dir']==direction)]
    return wave_window

def weather_window_get(time):
    weather_window = weather[(timeweather<time+datetime.timedelta(seconds=1)-time_shift_td) \
            & (timeweather>time+datetime.timedelta(seconds=1)-time_window_size_td-time_shift_td)]
    return weather_window

def wave_array_construct(wave_window,timestamp,milemarker,i):
    num_rows = int(space_window_size_mi/0.1)
    num_columns = int(time_window_size_min/1)
    wave_array = []
    if (len(wave_window)!=num_rows*num_columns):
        print('wave window size is wrong i=' + str(i) + ' ' + timestamp + ' ' + str(milemarker))
        return wave_array
    else:
        wave_array = np.zeros([num_rows,num_columns,3])
        mm_list = list(np.unique(wave_window['MM']))
        for i in range(len(mm_list)):
            mm = mm_list[i]
            wave_one_row = wave_window[wave_window['MM']==mm].sort_values('min', ascending=1)
            wave_array[i,:,0]=np.array(list(wave_one_row['speed']))
            wave_array[i,:,1]=np.array(list(wave_one_row['count']))
            wave_array[i,:,2]=np.array(list(wave_one_row['occup']))
        return wave_array
    
def weather_window_construct(weather_window,timestamp,milemarker,i):
    num_rows = int(space_window_size_mi/0.1)
    num_columns = int(time_window_size_min/1)
    weather_array = []
    weather_vector = np.zeros([num_columns,6])
    if (len(weather_window)!=num_columns):
        print('weather window size is wrong i=' + str(i) + ' ' + timestamp + ' ' + str(milemarker))
        return weather_array
    else:
        weather_window = weather_window.sort_values('time', ascending=0)
        weather_vector[:,0] = np.array(list(weather_window['tmpf']))
        weather_vector[:,1] = np.array(list(weather_window['dwpf']))
        weather_vector[:,2] = np.array(list(weather_window['tfs0']))
        weather_vector[:,3] = np.array(list(weather_window['tfs0_text']))
        weather_vector[:,4] = np.array(list(weather_window['tfs1']!='Dry')).astype('int')
        weather_vector[:,5] = np.array(list(weather_window['tfs1_text']!='Dry')).astype('int')
        
        for k in range(num_rows):
            weather_array.append(weather_vector)
        return np.asarray(weather_array)
    
def sample_label_get(i):
    
    sample = []
    label = []
    
    timestamp = waze['start'][i]
    milemarker = waze['MM'][i]
    month =timestamp[:2]
    day = timestamp[3:5]
    year = timestamp[6:10]
    date = year + '-' + month + '-' + day
    hour = int(timestamp[11:13])
    minite = int(timestamp[14:16])
    wavemin = hour*60+minite
    direction = int(waze['dir'][i]=='W')+1
    time = timewaze[i]
    my_file = Path('interp_daily/'+date+'.csv')
    if (my_file.is_file()):
        wave = pd.read_csv('interp_daily/'+date+'.csv',delimiter=',',header=None,names=['MM','dir','day','min','speed','count','occup'])
        wave_window = wave_window_get(wave,milemarker,wavemin,direction)
        wave_array = wave_array_construct(wave_window,timestamp,milemarker,i)
        weather_window = weather_window_get(time)
        weather_array = weather_window_construct(weather_window,timestamp,milemarker,i)
        if (len(wave_array)>0):
            sample = np.concatenate((wave_array,weather_array), axis=2)
            label = waze['TMCcrash'][i]
    return sample,label

In [ ]:
# single window

time_window_size_min = 10
space_window_size_mi = 1.
time_shift = 0
space_shift = 0.

time_window_size_td = datetime.timedelta(seconds=time_window_size_min*60)
time_shift_td = datetime.timedelta(seconds=time_shift*60)

samples =[]
labels = []
for i in range(len(waze)):
    sample,label = sample_label_get(i)
    if (len(sample)>0):
        samples.append(sample)
        labels.append(label)
samples = np.asarray(samples)
labels = np.asarray(labels)

In [4]:
# sliding windows

time_window_size_min = 5
space_window_size_mi = 1.
time_shifts = [0, 1, 2, 3, 4, 5] 
space_shifts = [0., -0.2, -0.4, -0.6, -0.8, -1.0] 

samples =[]
labels = []

for time_shift in time_shifts:
    for space_shift in space_shifts:
        
        time_window_size_td = datetime.timedelta(seconds=time_window_size_min*60)
        time_shift_td = datetime.timedelta(seconds=time_shift*60)
 
        for i in range(len(waze)):
            sample,label = sample_label_get(i)
            if (len(sample)>0):
                samples.append(sample)
                labels.append(label)
samples = np.asarray(samples)
labels = np.asarray(labels)
        
import pickle
pickle.dump( (samples,labels), open( "data_new.p", "wb" ) )

wave window size is wrong i=3 09/14/2016 16:52:00 13.64
wave window size is wrong i=13 09/14/2016 17:26:00 2.58
wave window size is wrong i=21 09/14/2016 17:58:00 13.66
wave window size is wrong i=24 09/14/2016 18:31:00 13.87
wave window size is wrong i=25 09/15/2016 07:30:00 1.97
wave window size is wrong i=26 09/15/2016 07:51:00 2.53
wave window size is wrong i=28 09/15/2016 07:49:00 2.38
wave window size is wrong i=31 09/15/2016 12:30:00 0.68
wave window size is wrong i=33 09/15/2016 16:31:00 13.48
wave window size is wrong i=35 09/15/2016 16:48:00 1.84
wave window size is wrong i=36 09/15/2016 16:49:00 2.14
wave window size is wrong i=38 09/15/2016 16:54:00 1.24
wave window size is wrong i=39 09/15/2016 17:10:00 0.6
wave window size is wrong i=40 09/15/2016 17:10:00 0.87
wave window size is wrong i=44 09/15/2016 17:22:00 2.31
wave window size is wrong i=45 09/15/2016 17:21:00 1.87
wave window size is wrong i=48 09/15/2016 17:25:00 1.46
wave window size is wrong i=52 09/16/2016 07:4

In [5]:
samples.shape


(73014, 10, 5, 9)

In [6]:
labels.shape

(73014,)

In [7]:
np.sum(labels)

7920

In [ ]:
# import pickle

# pickle.dump( (samples,labels), open( "data.p", "wb" ) )
#favorite_color = pickle.load( open( "save.p", "rb" ) )